In [ ]:
# 필요한 라이브러리 설치
!pip install -q kiwipiepy  #한국어 형태소 분석
!pip install -q wget       #HTTP, HTTPS 프로토콜을 사용하여 파일을 다운
!pip install -q sgmllib3k  # Python 2의 sgmllib를 Python 3에서 사용할 수 있게 해주는 라이브러리, SGML 파싱지원


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
import base64
import requests
from IPython.display import Image, display
import os
import time
from openai import OpenAI
from openai import AssistantEventHandler
from typing_extensions import override
from google.colab import userdata

# Open AI API Key 설정
openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
class MultiModal:
    def __init__(self, model, system_prompt=None, user_prompt=None):
        self.model = model
        self.system_prompt = system_prompt
        self.user_prompt = user_prompt
        self.init_prompt()

    def init_prompt(self):
        if self.system_prompt is None:
            self.system_prompt = "You are a helpful assistant who helps users to write a report related to images in Korean."
        if self.user_prompt is None:
            self.user_prompt = "Explain the images as an alternative text in Korean."

    # 이미지를 base64로 인코딩하는 함수 (URL)
    def encode_image_from_url(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            image_content = response.content
            if url.lower().endswith((".jpg", ".jpeg")):
                mime_type = "image/jpeg"
            elif url.lower().endswith(".png"):
                mime_type = "image/png"
            else:
                mime_type = "image/unknown"
            return f"data:{mime_type};base64,{base64.b64encode(image_content).decode('utf-8')}"
        else:
            raise Exception("Failed to download image")

    # 이미지를 base64로 인코딩하는 함수 (파일)
    def encode_image_from_file(self, file_path):
        with open(file_path, "rb") as image_file:
            image_content = image_file.read()
            file_ext = os.path.splitext(file_path)[1].lower()
            if file_ext in [".jpg", ".jpeg"]:
                mime_type = "image/jpeg"
            elif file_ext == ".png":
                mime_type = "image/png"
            else:
                mime_type = "image/unknown"
            return f"data:{mime_type};base64,{base64.b64encode(image_content).decode('utf-8')}"

    # 이미지 경로에 따라 적절한 함수를 호출하는 함수
    def encode_image(self, image_path):
        if image_path.startswith("http://") or image_path.startswith("https://"):
            return self.encode_image_from_url(image_path)
        else:
            return self.encode_image_from_file(image_path)

    def display_image(self, encoded_image):
        display(Image(url=encoded_image))

    def create_messages(
        self, image_url, system_prompt=None, user_prompt=None, display_image=True
    ):
        encoded_image = self.encode_image(image_url)
        if display_image:
            self.display_image(encoded_image)

        system_prompt = (
            system_prompt if system_prompt is not None else self.system_prompt
        )

        user_prompt = user_prompt if user_prompt is not None else self.user_prompt

        # 인코딩된 이미지를 사용하여 다른 처리를 수행할 수 있습니다.
        messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"{encoded_image}"},
                    },
                ],
            },
        ]
        return messages

    def invoke(
        self, image_url, system_prompt=None, user_prompt=None, display_image=True
    ):
        messages = self.create_messages(
            image_url, system_prompt, user_prompt, display_image
        )
        response = self.model.invoke(messages)
        return response.content

    def batch(
        self,
        image_urls: list[str],
        system_prompts: list[str] = [],
        user_prompts: list[str] = [],
        display_image=False,
    ):
        messages = []
        for image_url, system_prompt, user_prompt in zip(
            image_urls, system_prompts, user_prompts
        ):
            message = self.create_messages(
                image_url, system_prompt, user_prompt, display_image
            )
            messages.append(message)
        response = self.model.batch(messages)
        return [r.content for r in response]

    def stream(
        self, image_url, system_prompt=None, user_prompt=None, display_image=True
    ):
        messages = self.create_messages(
            image_url, system_prompt, user_prompt, display_image
        )
        response = self.model.stream(messages)
        return response


class OpenAIStreamHandler(AssistantEventHandler):
    @override
    def on_text_delta(self, delta, snapshot):
        return delta.value


class OpenAIAssistant:
    """
    OpenAI 어시스턴트를 관리하는 클래스입니다.
    이 클래스는 OpenAI API를 사용하여 파일 업로드, 어시스턴트 생성, 대화 관리 등의 기능을 제공합니다.
    """

    def __init__(self, configs):
        """
        OpenAIAssistant 클래스의 생성자입니다.

        :param configs: 설정 정보를 담은 딕셔너리
        configs = {
            "OPENAI_API_KEY": "OPENAI_API_KEY",
            "instructions": "사용자 입력 RAG 프롬프트미 설정시 기본 값",
            "PROJECT_NAME": "PDF-INTERVIEW-RAG-TEST", # 프로젝트 이름
            "model_name": "gpt-4o", # openai 모델 이름
            "chunk_size": 1000, # 청크 크기
            "chunk_overlap": 100, # 청크 중복 크기
        }
        """
        self.client = OpenAI(api_key=configs["OPENAI_API_KEY"])
        self.model = configs.get("model_name", "gpt-4o")
        self.instructions = configs.get("instructions", "")
        self.project_name = configs.get("PROJECT_NAME", "PDF-INTERVIEW-RAG-TEST")
        self.chunk_size = configs.get("chunk_size", 800)
        self.chunk_overlap = configs.get("chunk_overlap", 400)

        self.messages = []
        self.thread_id = None

    def upload_file(self, filepath):
        """
        파일을 OpenAI 서버에 업로드합니다.

        :param filepath: 업로드할 파일의 경로
        :return: 업로드된 파일의 ID
        """
        file = self.client.files.create(file=open(filepath, "rb"), purpose="assistants")
        return file.id

    def create_new_assistant(self, file_ids):
        """
        새로운 어시스턴트를 생성합니다.

        :param file_ids: 어시스턴트에 연결할 파일 ID 리스트
        :return: 생성된 어시스턴트의 ID와 벡터 스토어의 ID
        """
        # 현재 사용 사례에는 파일 검색 도구만 관련이 있습니다
        tools = [{"type": "file_search"}]

        chunking_strategy = {
            "type": "static",
            "static": {
                "max_chunk_size_tokens": self.chunk_size,
                "chunk_overlap_tokens": self.chunk_overlap,
            },
        }

        # 벡터 스토어 생성
        vector_store = self.client.beta.vector_stores.create(
            name=self.project_name,
            file_ids=file_ids,
            chunking_strategy=chunking_strategy,
        )
        tool_resources = {"file_search": {"vector_store_ids": [vector_store.id]}}

        # 어시스턴트 생성
        assistant = self.client.beta.assistants.create(
            name=self.project_name,
            instructions=self.instructions,
            model=self.model,
            tools=tools,
            tool_resources=tool_resources,
        )
        assistant_id = assistant.id
        vector_id = vector_store.id
        return assistant_id, vector_id

    def setup_assistant(self, assistant_id):
        """
        어시스턴트 ID를 설정합니다.

        :param assistant_id: 설정할 어시스턴트 ID
        """
        self.assistant_id = assistant_id

    def setup_vectorstore(self, vector_id):
        """
        벡터 스토어 ID를 설정합니다.

        :param vector_id: 설정할 벡터 스토어 ID
        """
        self.vector_id = vector_id

    def _start_assistant_thread(self, prompt):
        """
        어시스턴트와의 대화 스레드를 시작합니다.

        :param prompt: 초기 프롬프트 메시지
        :return: 생성된 스레드의 ID
        """
        # 메시지 초기화
        self.messages = [{"role": "user", "content": prompt}]

        # 스레드 생성
        tool_resources = {"file_search": {"vector_store_ids": [self.vector_id]}}
        thread = self.client.beta.threads.create(
            messages=self.messages, tool_resources=tool_resources
        )

        return thread.id

    def _run_assistant(self, thread_id):
        """
        어시스턴트를 실행합니다.

        :param thread_id: 실행할 스레드의 ID
        :return: 실행된 작업의 ID
        """
        run = self.client.beta.threads.runs.create(
            thread_id=thread_id, assistant_id=self.assistant_id
        )
        return run.id

    def _check_run_status(self, thread_id, run_id):
        """
        실행 상태를 확인합니다.

        :param thread_id: 스레드 ID
        :param run_id: 실행 ID
        :return: 실행 상태
        """
        run = self.client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        return run.status

    def _retrieve_thread_messages(self, thread_id):
        """
        스레드의 메시지를 검색합니다.

        :param thread_id: 검색할 스레드의 ID
        :return: 메시지 리스트
        """
        thread_messages = self.client.beta.threads.messages.list(thread_id)
        list_messages = thread_messages.data
        thread_messages = []
        for message in list_messages:
            obj = {}
            obj["content"] = message.content[0].text.value
            obj["role"] = message.role
            thread_messages.append(obj)
        return thread_messages[::-1]

    def _add_messages_to_thread(self, thread_id, user_message):
        """
        스레드에 새 메시지를 추가합니다.

        :param thread_id: 메시지를 추가할 스레드의 ID
        :param user_message: 추가할 사용자 메시지
        :return: 추가된 메시지 객체
        """
        thread_message = self.client.beta.threads.messages.create(
            thread_id, role="user", content=user_message
        )
        return thread_message

    def invoke(self, message):
        """
        어시스턴트에게 메시지를 보내고 응답을 받습니다.

        :param message: 보낼 메시지
        :return: 어시스턴트의 응답
        """
        if len(self.messages) == 0:
            self.thread_id = self._start_assistant_thread(message)
        else:
            self._add_messages_to_thread(self.thread_id, message)

        run_id = self._run_assistant(self.thread_id)
        while self._check_run_status(self.thread_id, run_id) != "completed":
            time.sleep(1)
        answer = self._retrieve_thread_messages(self.thread_id)
        return answer[-1]["content"]

    def stream(self, message):
        """
        어시스턴트에게 메시지를 보내고 응답을 스트림으로 받습니다.

        :param message: 보낼 메시지
        :return: 어시스턴트의 응답 스트림
        """
        if len(self.messages) == 0:
            self.thread_id = self._start_assistant_thread(message)
        else:
            self._add_messages_to_thread(self.thread_id, message)

        handler = OpenAIStreamHandler()

        with self.client.beta.threads.runs.stream(
            thread_id=self.thread_id,
            assistant_id=self.assistant_id,
            instructions=self.instructions,
            event_handler=handler,
        ) as stream:
            for text in stream.text_deltas:
                yield text

    def list_chat_history(self):
        """
        대화 기록을 반환합니다.

        :return: 대화 기록 리스트
        """
        return self._retrieve_thread_messages(self.thread_id)

    def clear_chat_history(self):
        """
        대화 기록을 초기화합니다.
        """
        self.messages = []
        self.thread_id = None

In [ ]:
# RAG 시스템 프롬프트 입력
_DEFAULT_RAG_INSTRUCTIONS = """업로드된 자기소개서의 내용을 기반으로 엄격한 압박 면접을 진행하는 면접관 역할을 맡아주세요. 실제 면접처럼 한가지의 질문 혹은 요청을 합니다.

1. 시작 인사 및 첫 질문
- "안녕하세요, 000지원자님. 면접 시작하겠습니다. 먼저 간단하게 자기소개 부탁드립니다."
- 자기소개 또는 지원동기에서 드러난 주요 포인트를 파악하고 이후 질문을 준비합니다.

2. 직무 관련 질문
- 지원 직무와 자기소개서의 주요 경력 또는 경험에 대해 구체적인 질문을 2개 진행합니다.
- 지원자가 경험한 상황, 수행한 역할, 결과에 대해 세부적으로 묻고 검증합니다.
- 질문은 지원 직무와 자기소개서 내용 간의 연관성을 평가하는 방향으로 진행합니다.

3. 연계 질문
- 직무 관련 질문에서 나온 답변을 바탕으로 1개의 연계 질문을 던집니다.
- 자기소개서에 포함되지 않은 전공 지식, 산업 트렌드, 직무 전문성을 평가할 수 있는 심화 질문을 포함합니다.
- 답변이 자기소개서와 일치하는지 확인하며, 모호한 부분이 있다면 추가 설명을 요청합니다.

4. 압박 질문
- 지원자가 준비하지 못했을 가능성이 높은 예상치 못한 질문이나 논리적 사고를 요구하는 질문을 1~2개 추가합니다.
- 답변이 부족하거나 모호한 경우, 근거를 요구하거나 대안을 묻는 방식으로 압박감을 유지합니다.
- 답변이 직무와 직접적으로 연관되는지 확인하며, 실무에서의 적용 가능성에 대해 탐구합니다.

5. 결론 및 마무리
- 마지막 질문을 할 때, "이제 마지막 질문입니다."라고 명확히 알립니다.
- 마지막 질문에 대한 답변이 끝나면 추가 질문 없이 "오늘 면접에 참여해주셔서 감사합니다. 앞으로의 과정에서도 좋은 결과 있기를 바랍니다."라는 인사로 마무리합니다.

# Output Format
- 질문을 할 때마다 한 번에 하나씩 간결하게 제시합니다.
- 추가 질문이 필요할 경우, 연계 질문을 통해 대화를 이어갑니다.
- 마지막 질문 후에는 간단한 마무리 인사로 면접을 종료합니다.

# Notes
- 자연스러운 흐름을 유지하며 논리적으로 질문을 전개합니다.
- 가능한 한 압박적인 질문을 지속하며, 진정성 있는 답변을 유도합니다.
- 답변의 모호함이나 준비되지 않은 부분을 의도적으로 도전하는 질문을 던집니다.
- 심화 질문을 통해 지원자의 전공 및 직무 관련 지식을 평가합니다.
- 모든 대화는 한국어로 진행하며, 매 순간 하나의 질문만 던집니다.
- 마지막 질문 이후에는 추가 질문 없이 면접을 종료합니다.
"""
# 설정(configs)
configs = {
    "OPENAI_API_KEY": openai_api_key,  # OpenAI API 키
    "instructions": _DEFAULT_RAG_INSTRUCTIONS,  # RAG 시스템 프롬프트
    "PROJECT_NAME": "INTERVIEW-TEST",  # 프로젝트 이름(자유롭게 설정)
    "model_name": "gpt-4o",  # 사용할 OpenAI 모델 이름(gpt-4o, gpt-4o-mini, ...)
    "chunk_size": 1000,  # 청크 크기
    "chunk_overlap": 100,  # 청크 중복 크기
}


# 인스턴스 생성
assistant = OpenAIAssistant(configs)

In [ ]:
# 업로드할 파일 경로
data = "자기소개서_샘플.pdf"

# 파일 업로드 후 file_id 는 잘 보관해 두세요. (대시보드에서 나중에 확인 가능)
file_id = assistant.upload_file(data)

In [ ]:
# 업로드한 파일의 ID 리스트 생성
file_ids = [file_id]

# 새로운 어시스턴트 생성 및 ID 받기
assistant_id, vector_id = assistant.create_new_assistant(file_ids)

# 어시스턴트 설정
assistant.setup_assistant(assistant_id)

# 벡터 스토어 설정
assistant.setup_vectorstore(vector_id)


In [ ]:
##이미 있는 assistant_id, vector_id 가 있으면 아래 코드로 실행
# assistant_id = "asst_~~~~........"
# vector_id = "vs_~~~~........"

# # 어시스턴트 설정
# assistant.setup_assistant(assistant_id)

# # 벡터 스토어 설정
# assistant.setup_vectorstore(vector_id)

In [ ]:
#대화 stream(),invoke() 중 stream()으로 구
for token in assistant.stream("안녕하세요. 정현정입니다."):
    print(token, end="", flush=True)

안녕하세요, 정현정 지원자님. 면접 시작하겠습니다. 먼저 간단하게 자기소개 부탁드립니다.

In [ ]:
for token in assistant.stream("엔지니어 직무에 지원하게 된 동기는 아무나 할 수 없는 특수성 때문입니다. 군 복무 중 정전이 발생한 적이 있습니다. 당직근무를 서고 있던 저는 전기 군무원님과 함께 부대 내의 수변전실로 들어갔습니다. 비상발전기, LBS, VCB 등 다양한 전기설비를 능숙하게 다루어 정전을 해결하는 것을 보고 전기 엔지니어의 꿈을 가지게 되었습니다.입사 후 목표는 디지털 전기 엔지니어가 되는 것입니다."):
    print(token, end="", flush=True)

정현정 지원자님, 군 복무 시 경험하신 정전 해결 과정에서 어떤 역할을 하셨는지 구체적으로 설명해 주실 수 있나요? 예를 들어, 그 상황에서 어떤 특별한 기여를 하셨는지, 어떤 문제 해결 과정을 거치셨는지 궁금합니다.

In [ ]:
for token in assistant.stream("복무 중 정전 상황에서는 당직근무자로서 초기 대응을 맡았습니다. 정전이 발생했을 때, 우선 부대 내 주요 설비의 상태를 확인하고 전기 군무원님과 함께 문제 해결 절차에 참여했습니다. 이 과정에서 수변전실로 이동해 비상발전기와 LBS(부하개폐기), VCB(진공차단기)와 같은 전기 설비의 동작 원리를 배울 수 있었습니다. 당시 제가 맡았던 역할은 군무원님이 지시하는 대로 각 설비의 상태를 점검하고, 적절히 조작하며 데이터를 기록하는 일이었습니다. 또한, 비상발전기의 작동을 확인하고 정전 구간을 빠르게 분리하는 작업에 직접 참여했습니다.이 경험은 전기 엔지니어가 단순히 설비를 다루는 것을 넘어, 체계적인 문제 해결과 신속한 판단이 필요한 직무라는 점을 깨닫게 해주었습니다. 이후 이 분야에 대한 관심이 깊어졌고, 특히 디지털 기술이 접목된 전기 설비와 자동화 시스템에 대한 열정을 키우게 되었습니다. 이 경험은 전기 엔지니어로서의 꿈을 더욱 구체화하고, 미래에 내가 맡게 될 역할이 어떤 책임감과 전문성을 요구하는지 실감할 수 있는 계기가 되었습니다."):
    print(token, end="", flush=True)

감사합니다. 이제 직무와 관련된 질문 드리겠습니다. 전기 엔지니어로서 디지털 기술이 접목된 전기 설비와 자동화 시스템에 관심을 가지셨다고 하셨습니다. 그러한 시스템을 개발하거나 관리하기 위해 필요한 핵심 기술이나 지식은 무엇이라고 생각하시나요? 또한, 그 중 어떤 부분에 대해 현재까지 공부하거나 경험하신 것이 있는지 구체적으로 설명 부탁드립니다.

In [ ]:
for token in assistant.stream("디지털 기술이 접목된 전기 설비와 자동화 시스템을 개발하거나 관리하기 위해 필요한 핵심 기술로는 전력 시스템의 설계 및 제어 지식, 프로그래밍과 IoT 기술, 그리고 데이터 분석 및 AI 활용 능력을 들 수 있습니다. 첫째, 전력 설비와 자동화 시스템의 안정성과 효율성을 보장하기 위해 전력 설비의 작동 원리와 제어 이론, PLC 및 SCADA와 같은 제어 시스템에 대한 지식이 필수적입니다. 둘째, 자동화 시스템을 설계하려면 하드웨어와 소프트웨어 간의 상호작용을 구현할 수 있는 프로그래밍 기술이 중요하며, Python이나 C와 같은 언어는 IoT 기반 센서 데이터 수집 및 제어 시스템 개발에 자주 활용됩니다. 셋째, 디지털화된 시스템에서는 대량의 데이터를 수집하고 이를 분석해 이상 징후를 감지하거나 시스템의 효율성을 최적화해야 하므로 빅데이터 분석과 머신러닝 알고리즘 활용 능력도 필요합니다. 저는 이 중 프로그래밍과 IoT 기술 분야에서 경험을 쌓아왔습니다. 예를 들어, 2019년 캡스톤 디자인 경진대회에서 라즈베리파이를 이용한 반려동물 자동 급식 시스템을 설계하며 Python 프로그래밍을 활용해 센서 데이터를 수집하고 하드웨어 동작을 제어하는 로직을 설계한 경험이 있습니다. 이러한 경험을 통해 IoT 기반 시스템 설계의 기본 원리를 이해했으며, 이와 함께 Pandas와 같은 도구를 활용해 데이터 분석 기초도 학습해왔습니다. 앞으로는 전력 설비와 제어 이론, 그리고 데이터 분석 기술을 더욱 심화 학습하여 실무에서 이 기술들을 통합적으로 활용할 수 있도록 준비해 나가겠습니다."):
    print(token, end="", flush=True)

앞으로의 학습 계획과 관련하여, 전력 설비나 제어 이론에 대해 심화 학습을 하고 싶으시다고 하셨습니다. 그렇다면, 현재까지 이러한 분야에서 참조하신 자료나 특정 연구 주제가 있다면 소개해 주실 수 있나요? 그리고 이와 관련된 산업 트렌드나 기술 발전에 대한 의견도 함께 부탁드립니다.

In [ ]:
for token in assistant.stream("현재까지 전력 설비와 제어 이론에 대한 학습을 위해 참조한 주요 자료로는 전력 공학의 기본서와 IEEE Xplore에서 제공하는 논문들을 활용했습니다. 예를 들어, 스마트 그리드 기술과 에너지 저장 시스템(ESS)의 통합을 다룬 연구나 SCADA와 PLC를 기반으로 한 제어 시스템 설계 자료를 통해 이론적 기초를 쌓았습니다. 특히, 스마트 그리드와 관련된 논문은 실시간 전력 수요 예측과 자원 분배의 효율성을 높이는 데 필요한 디지털 기술의 활용 방식을 이해하는 데 큰 도움이 되었습니다. 산업 트렌드 측면에서 보면, 최근 스마트 그리드 기술은 AI와 빅데이터를 결합하여 전력망의 안정성과 유연성을 높이는 방향으로 발전하고 있으며, 초고압 직류송전(HVDC) 기술은 장거리 전력 전송의 손실을 줄이고 재생에너지 통합을 지원하는 핵심 기술로 자리 잡고 있습니다. 또한, 디지털 트윈 기술은 전력 설비의 실시간 모니터링과 시뮬레이션을 가능하게 하여 유지보수와 운영 효율성을 크게 개선하고 있습니다. 이러한 기술 발전은 전력 설비와 디지털 기술의 융합을 가속화하고 있으며, 앞으로도 AI 기반 제어 시스템과 데이터 분석 기술이 전력 엔지니어링의 핵심으로 자리 잡을 것으로 보입니다. 저는 이러한 트렌드에 맞춰 전력 설비 설계와 디지털 기술 융합에 대한 심화 학습을 이어가고, 시뮬레이션 도구를 활용한 설계 실습과 프로젝트 참여를 통해 실무 경험을 쌓아 나가겠습니다. 이를 통해 회사의 기술 혁신에 기여할 수 있는 엔지니어로 성장하고자 합니다."):
    print(token, end="", flush=True)

광범위한 자료를 바탕으로 심도 있게 학습하고 계시군요. 이제 마지막 질문 드리겠습니다. 만약, 디지털 기술을 이용한 전력 설비 자동화 과정에서 예상하지 못한 문제가 발생했을 경우, 이를 해결하기 위한 첫 번째 단계는 무엇이라고 생각하시나요? 그리고 그러한 문제를 해결하기 위해 어떤 정보를 수집하거나 분석할 것인지 간단히 설명 부탁드립니다.

In [ ]:
for token in assistant.stream("디지털 기술을 이용한 전력 설비 자동화 과정에서 예상하지 못한 문제가 발생했을 경우, 이를 해결하기 위한 첫 번째 단계는 문제의 원인을 정확히 파악하는 것이라고 생각합니다. 이를 위해 설비의 동작 로그와 센서 데이터를 분석하고, 문제가 발생한 시점과 관련된 조건을 명확히 확인해야 합니다. 예를 들어, 시스템의 입력값과 출력값, 네트워크 상태, 설비 간 통신 여부 등을 점검하여 문제의 원인이 하드웨어 고장인지, 소프트웨어 오류인지, 데이터 통신 문제인지 파악합니다. 정보 수집 측면에서는 첫째로 자동화 시스템의 로그 데이터와 알람 기록을 분석해 오류가 발생한 위치와 시간대를 확인하며, 둘째로 센서 및 제어 장치의 실시간 데이터를 모니터링해 정상 범위를 벗어난 지점을 찾고, 셋째로 최근 소프트웨어 업데이트나 네트워크 변경 사항 등 시스템 환경에서의 변화를 확인하여 문제가 발생한 배경을 종합적으로 분석합니다. 이러한 데이터를 바탕으로 문제를 구조화하여 잠재적 원인과 해결 방안을 도출할 수 있습니다. 문제 해결의 핵심은 체계적인 데이터 분석과 원인 분류에 있으며, 이를 통해 신속하고 논리적으로 대응해 시스템의 안정성을 유지하는 엔지니어로 성장하고자 합니다."):
    print(token, end="", flush=True)

오늘 면접에 참여해주셔서 감사합니다. 앞으로의 과정에서도 좋은 결과 있기를 바랍니다.

In [ ]:
# 대화 목록 조회
assistant.list_chat_history()

[{'content': '안녕하세요. 정현정입니다.', 'role': 'user'},
 {'content': '안녕하세요, 정현정 지원자님. 면접 시작하겠습니다. 먼저 간단하게 자기소개 부탁드립니다.',
  'role': 'assistant'},
 {'content': '엔지니어 직무에 지원하게 된 동기는 아무나 할 수 없는 특수성 때문입니다. 군 복무 중 정전이 발생한 적이 있습니다. 당직근무를 서고 있던 저는 전기 군무원님과 함께 부대 내의 수변전실로 들어갔습니다. 비상발전기, LBS, VCB 등 다양한 전기설비를 능숙하게 다루어 정전을 해결하는 것을 보고 전기 엔지니어의 꿈을 가지게 되었습니다.입사 후 목표는 디지털 전기 엔지니어가 되는 것입니다.',
  'role': 'user'},
 {'content': '정현정 지원자님, 군 복무 시 경험하신 정전 해결 과정에서 어떤 역할을 하셨는지 구체적으로 설명해 주실 수 있나요? 예를 들어, 그 상황에서 어떤 특별한 기여를 하셨는지, 어떤 문제 해결 과정을 거치셨는지 궁금합니다.',
  'role': 'assistant'},
 {'content': '복무 중 정전 상황에서는 당직근무자로서 초기 대응을 맡았습니다. 정전이 발생했을 때, 우선 부대 내 주요 설비의 상태를 확인하고 전기 군무원님과 함께 문제 해결 절차에 참여했습니다. 이 과정에서 수변전실로 이동해 비상발전기와 LBS(부하개폐기), VCB(진공차단기)와 같은 전기 설비의 동작 원리를 배울 수 있었습니다. 당시 제가 맡았던 역할은 군무원님이 지시하는 대로 각 설비의 상태를 점검하고, 적절히 조작하며 데이터를 기록하는 일이었습니다. 또한, 비상발전기의 작동을 확인하고 정전 구간을 빠르게 분리하는 작업에 직접 참여했습니다.이 경험은 전기 엔지니어가 단순히 설비를 다루는 것을 넘어, 체계적인 문제 해결과 신속한 판단이 필요한 직무라는 점을 깨닫게 해주었습니다. 이후 이 분야에 대한 관심이 깊어졌고, 특히 디지털 기술이 접목된 전기 설비와 자동화 시스템에 대한

In [ ]:
# 대화 초기화
assistant.clear_chat_history()